In [1]:
import os
from pathlib import Path
# Change cwd to the project root (parent of 'notebooks/')
os.chdir(Path.cwd().parent)
Path.cwd()

PosixPath('/Users/jbrandt/code/birddog')

In [2]:
from birddog.core import Archive, form_element_text
from birddog.utility import fetch_url, form_text_item, convert_utc_time
from birddog.wiki import history_url, get_pages_history, get_page_history, get_page_history_from_cutoff, HistoryLRU, ARCHIVES, ARCHIVE_LIST
from urllib.parse import quote
#import mwclient
from bs4 import BeautifulSoup

2025-04-17 17:45:51,533 [INFO] Found credentials in environment variables.
2025-04-17 17:45:51,618 [INFO] Using AWS S3 bucket birddog-data for storage.


ImportError: cannot import name 'get_pages_history' from 'birddog.wiki' (/Users/jbrandt/code/birddog/birddog/wiki.py)

In [3]:
archive = Archive('DAZHO')
archive.default_url

2025-04-17 17:45:52,744 [INFO] HistoryLRU.lookup(ДАЖО/Д): cache miss
2025-04-17 17:45:52,985 [INFO] fetch_url: 1 requests in last 60s → 0.02 req/s
2025-04-17 17:45:53,667 [INFO] Retrieved from cache: DAZHO-D[2025,01,19,12:47]: page_cache/DAZHO-D/2025,01,19,12:47.json


'https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94'

In [4]:
archive.url

'https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94'

In [5]:
archive.unquoted_url

'https://uk.wikisource.org/wiki/Архів:ДАЖО/Д'

In [ ]:
get_pages_history([archive.title], limit=5)

In [ ]:
fond = archive.lookup('1')
fond.title

In [ ]:
get_pages_history([fond.title], limit=5)

In [ ]:
get_pages_history([fond.title, archive.title], limit=5)

In [ ]:
from urllib.parse import unquote

In [ ]:
unquote(fond._spec[1].split(":")[1])

In [ ]:
archive.title

In [ ]:
[archive.title, fond.title]

In [ ]:
history_url([archive.title, fond.title])

In [ ]:
WIKI_NAMESPACE  = 'Архів'
def wiki_title(page_title):
    return f'{WIKI_NAMESPACE}:{page_title}'

def history_url2(page_titles, limit=1):
    if isinstance(page_titles, str):
        titles_param = quote(wiki_title(page_titles))
    else:
        titles_param = '|'.join([quote(wiki_title(title)) for title in page_titles])

    return (
        'https://uk.wikisource.org/w/api.php?action=query&format=json'
        '&prop=revisions&rvprop=ids|timestamp'
        f'&titles={titles_param}'
    )


In [ ]:
history_url2([fond.title, archive.title])

In [ ]:
s='https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94'

In [ ]:
unquote(s)

In [ ]:
fond.history(limit=10)

In [ ]:
fond.id

In [ ]:
archive.parent

In [ ]:
archive._archive_name

In [ ]:
p = Archive('DAZHO', subarchive='P')

In [ ]:
p._archive_name

In [ ]:
get_page_history('xyz')

In [ ]:
get_page_history_from_cutoff(archive.title, cutoff_date='2025,03,01')

In [ ]:
lru=HistoryLRU()

In [ ]:
lru.lookup(archive.title, limit=5)

In [ ]:
lru.lookup(archive.title, limit=5)

In [ ]:
lru.lookup(archive.title, limit=3)

In [ ]:
lru.lookup(archive.title, limit=15)

In [ ]:
lru.lookup_by_cutoff(archive.title, cutoff_date='2024')

In [ ]:
ARCHIVE_LIST["DAZHO"]

In [ ]:
ARCHIVES["DAZHO"]

In [ ]:
archive.title

In [ ]:
p = Archive('DAZHO','Р')

In [ ]:
p.title

In [ ]:
p._archive_name

In [ ]:
def content_url(page):
    title = 'Архів:' + page.title.replace('/', '%2F')
    print(title)
    return f'https://en.wikisource.org/w/api.php?action=parse&format=json&page={title}&prop=text'

In [ ]:
def content_url(revid):
    return f'https://uk.wikisource.org/w/api.php?action=parse&format=json&oldid={revid}'

In [ ]:
pg = fetch_url(content_url(871572), json=True)
pg = pg['parse']
print(f'{pg["title"]}: revid={pg["revid"]}')
soup = BeautifulSoup(pg['text']['*'])
print(archive.title)

In [ ]:
def read_page(soup):
    """
    Extract archive information for given page using HTTP get.
    Return struct with page:
        title,
        description,
        table header,
        table contents,
        lastmod date,
        doc_link [only for case pages],
        thumb_link [only for case pages],
    """
    title_text = soup.find('span', attrs = {'id': 'header_title_text'}).split(':')
    namespace = title_text[0]
    title = title_text[1]
    desc = soup.find('span', attrs = {'id': 'header_section_text'})
    table = soup.find('table', attrs = {'class': 'wikitable'})
    children = []
    header = []
    if table:
        for tr_elem in table.find_all('tr'):
            if not header:
                for th_elem in tr_elem.find_all('th'):
                    header.append(form_element_text(th_elem))
            item = []
            for td_elem in tr_elem.find_all('td'):
                a_elem = td_elem.find('a')
                child_url = a_elem.get('href') if a_elem else None
                text = form_text_item(td_elem.text.strip())
                item.append({'text': text, 'link': child_url})
            if item:
                children.append(item)

    # check for document thumbnail
    doc_info = soup.find('figure', attrs = {'typeof': 'mw:File/Thumb'})
    doc_url = None
    thumb_url = None
    if doc_info:
        a_tag = doc_info.find('a')
        doc_url = a_tag.get('href')
        thumb_elem = a_tag.find('img')
        thumb_url = thumb_elem.get('src') if thumb_elem else None
    footer = soup.find('li', attrs={'id': 'footer-info-lastmod'})
    last_modified = lastmod(footer.text) if footer else None

    return {
        'title': form_element_text(title),
        'namespace': form_element_text(namespace),
        'description': form_element_text(desc),
        'header': header,
        'children': children,
        'lastmod': last_modified,
        #'link': url,
        'doc_link': doc_url,
        'thumb_link': thumb_url,
    }


In [ ]:
archive._page['title']

In [ ]:
with open('tmp.html','w') as file:
    file.write(str(soup))

In [ ]:
title = soup.find('span', attrs = {'id': 'header_title_text'})

In [ ]:
title.text

In [ ]:
desc = soup.find('span', attrs = {'id': 'header_section_text'})
desc

In [ ]:
page = read_page(soup)

In [ ]:
page.keys()

In [ ]:
soup

In [ ]:
archive.title.replace('/','%2F')

In [ ]:
site = mwclient.Site('uk.wikisource.org')

In [ ]:
page = site.pages['Архів:ДАЖО/Д']
text = page.text()

In [ ]:
page.revisions()

In [ ]:
for r in page.revisions():
    print(r)

In [ ]:
for rev in page.revisions(limit=5, api_chunk_size=None):
    print(rev)

In [ ]:
p2 = site.pages['624167']

In [ ]:
all_titles = [p.page_title for p in site.allpages(max_items=8000)]

In [ ]:
all_titles[-1]

In [ ]:
https://uk.wikisource.org/w/api.php?action=parse&format=json&oldid=622582

In [ ]:
page.